# Phase 0: UFoE 5-필터 검증

PDB 대표 구조(1,478개 목표)에 5개 기하학적 필터 적용, **80%+ 통과율** 확인.

In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

import pandas as pd
from src.filters.ufoef_filters import apply_all_filters, batch_validate
from src.utils.pdb_parser import load_structure

## 1. PDB 목록 준비

`data/pdb_representative/` 아래에 PDB 파일을 두거나, 경로 리스트를 직접 지정.

In [ ]:
pdb_dir = ROOT / "data" / "pdb_representative"
pdb_list = list(pdb_dir.glob("*.pdb")) + list(pdb_dir.glob("*.cif"))
pdb_list = [str(p) for p in pdb_list]
print(f"PDB 파일 수: {len(pdb_list)}")
if len(pdb_list) == 0:
    print("예: data/pdb_representative/ 에 PDB를 넣거나 아래 pdb_list를 수동으로 채우세요.")

## 2. 배치 검증 실행

In [ ]:
if len(pdb_list) > 0:
    df = batch_validate(pdb_list)
    print(df.head())
    print("\n--- 통과율 ---")
    pass_rate = df["all_passed"].mean() * 100
    print(f"전체 통과(all_passed): {pass_rate:.1f}% ({df['all_passed'].sum()} / {len(df)})")
    for col in df.columns:
        if col.endswith("_passed"):
            print(f"  {col}: {df[col].mean()*100:.1f}%")
else:
    df = pd.DataFrame()
    print("PDB 목록이 비어 있어 배치 검증을 건너뜁니다.")

## 3. 검증 기준

- **목표: 80%+** 구조가 5개 필터 모두 통과 (Primary 임계값).
- 통과율이 80% 미만이면 제약 조건 정의 또는 파싱 이슈 점검.

In [ ]:
if len(df) > 0:
    target_ok = pass_rate >= 80.0
    print(f"80%+ 통과 목표: {'달성' if target_ok else '미달성'}")